## Visualizing the queries
* When you've got your data indexed to Solr, it is nice to be able to show the results to others as well
* the browse page works pretty well http://localhost:8900/solr/reddit/browse
* Problem: Very unsecure, meant for just prototyping

* To solve this we use Blacklight, a frontend software that interacts with your Solr
* http://localhost:3000

## Installing Blacklight

* Requires Ruby and Rails

In [1]:
##Installing the Solr wrapper
gem install solr_wrapper

Fetching: solr_wrapper-0.23.0.gem (100%)
Parsing documentation for solr_wrapper-0.23.0
Installing ri documentation for solr_wrapper-0.23.0
Done installing documentation for solr_wrapper after 0 seconds
1 gem installed


In [2]:
## Creating a blacklight project called 'Solr_frontend' (You can choose whatever name you want)
rails new Solr_frontend -m https://raw.github.com/projectblacklight/blacklight/master/template.demo.rb

      create  
      create  README.md
      create  Rakefile
      create  config.ru
      create  .gitignore
      create  Gemfile
      create  app
      create  app/assets/config/manifest.js
      create  app/assets/javascripts/application.js
      create  app/assets/javascripts/cable.js
      create  app/assets/stylesheets/application.css
      create  app/channels/application_cable/channel.rb
      create  app/channels/application_cable/connection.rb
      create  app/controllers/application_controller.rb
      create  app/helpers/application_helper.rb
      create  app/jobs/application_job.rb
      create  app/mailers/application_mailer.rb
      create  app/models/application_record.rb
      create  app/views/layouts/application.html.erb
      create  app/views/layouts/mailer.html.erb
      create  app/views/layouts/mailer.text.erb
      create  app/assets/images/.keep
      create  app/assets/javascripts/channels
      create  app/assets/javascripts/channels/.keep
      create 

* We have now successfully installed Blacklight
* Now we just have to configure Blacklight to understand our data as well as modify our Solr a bit

## Configuring Solr

* For Solr, we need to define two new SearchHandlers that Blacklight expects and change few other lines
* Custom SearchHandlers are basically aliases or shortcuts for certain type of queries
* For example we could query our data using:
    * http://127.0.0.1:8900/solr/reddit/select?q=body:dog&rows=100&facet=on&facet.field=subreddit
* Or we can define a custom SearchHandler that does all of that
    * http://127.0.0.1:8900/solr/reddit/select?qt=find100dogs
* These are defined in the solrconfig.xml file
* Unfortunately there's no graphical interface for this, so we have to do it manually

In [1]:
cd /home/avjves/solr-6.4.2/server/solr/reddit/conf
cat solrconfig.xml

<?xml version="1.0" encoding="UTF-8" ?>
<!--
 Licensed to the Apache Software Foundation (ASF) under one or more
 contributor license agreements.  See the NOTICE file distributed with
 this work for additional information regarding copyright ownership.
 The ASF licenses this file to You under the Apache License, Version 2.0
 (the "License"); you may not use this file except in compliance with
 the License.  You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
-->

<!-- 
     For more details about configurations options that may appear in
     this file, see http://wiki.apache.org/solr/SolrConfigXml. 
-->
<config>
  <!-- In all configuration be

* Here we have lots of settings for our core
* So what we have to do here is to allow custom handlers, define the two new handlers and delete an overriding handler

* <b> 1st step: </b>
    * Find a line starting with '&lt;requestDispatcher'
    * It should be &lt;requestDispatcher handleSelect="false"&gt; by default
    * change that false to true
    * This allows Solr to have custom handlers
* <b> 2nd step: </b>
    * Find the line &lt;/requestDispatcher&gt;
    * We can now define the custom handlers starting AFTER this line
    * Custom handlers are defined like this:
    * &lt;requestHandler name="custom_handler_name" class="solr.SearchHandler"&gt; *handler specifications here* &lt;/requestHandler&gt;
    
    * Now we need to define a custom handler that has the name "document", finds all fields, returns 1 row, and find a document based on its ID
    

In [ ]:
  <requestHandler name="document" class="solr.SearchHandler" >
    <lst name="defaults">
      <str name="echoParams">all</str>   ## Shows query settings in the result
      <str name="fl">*</str> ## Finding all fields
      <str name="rows">1</str> ## Just 1 row
      <str name="q">{!term f=id v=$id}</str> ## querying from field id, with value $id that is received from Blacklight
    </lst>
  </requestHandler>

* <b> 3rd step: </b>
    * One more handler to add
    * This is a bit more complex. This is used to search for all results

In [ ]:
<requestHandler name="search" class="solr.SearchHandler" default="true">
     <lst name="defaults">
       <str name="defType">dismax</str> ##Use dismax query parser
       <str name="echoParams">explicit</str> ## Show query settings in the result
       <int name="rows">10</int> ## Return 10 rows

       <str name="q.alt">*:*</str> ## Alternative query to find everything. This is used if no query was given

       <str name="qf"> ## Which fields to query against. This is used if we use the "All fields" option when searching
                       ## with blacklight
         body
         subreddit
       </str>
       <str name="text_qf"> ## Only search from the text field if using the "Text" option
         body
       </str>
       <str name="subreddit_qf"> ## Only search from the subreddit field if using the "Subreddit" option
         subreddit
       </str>
       <str name="fl">id,body,subreddit,ups,downs,gilded</str> ## Which fields to return

       <str name="facet">true</str> ## Facets on
       <str name="facet.mincount">1</str> ## To show a facet, it needs to have at least 1 hit
       <str name="facet.field">subreddit</str> ## Which field to use as facets

     </lst>
  </requestHandler>

So basically you can just copy these two handlers AFTER the &lt;/requestDispatcher&gt; line

<b> 4th step </b>
   * Remove the /select requestHandler to stop it from overwriting our custom handlers
   * it should look something like this:


In [ ]:
<requestHandler name="/select" class="solr.SearchHandler">
    <!-- default values for query parameters can be specified, these
         will be overridden by parameters in the request
      -->
    <lst name="defaults">
      <str name="echoParams">explicit</str>
      <int name="rows">10</int>
      <!-- <str name="df">text</str> -->
    </lst>
    <!-- In addition to defaults, "appends" params can be specified
         to identify values which should be appended to the list of
         multi-val params from the query (or the existing "defaults").
      -->
    <!-- In this example, the param "fq=instock:true" would be appended to
         any query time fq params the user may specify, as a mechanism for
         partitioning the index, independent of any user selected filtering
         that may also be desired (perhaps as a result of faceted searching).

         NOTE: there is *absolutely* nothing a client can do to prevent these
         "appends" values from being used, so don't use this mechanism
         unless you are sure you always want it.
      -->
    <!--
       <lst name="appends">
         <str name="fq">inStock:true</str>
       </lst>
      -->
    <!-- "invariants" are a way of letting the Solr maintainer lock down
         the options available to Solr clients.  Any params values
         specified here are used regardless of what values may be specified
         in either the query, the "defaults", or the "appends" params.

         In this example, the facet.field and facet.query params would
         be fixed, limiting the facets clients can use.  Faceting is
         not turned on by default - but if the client does specify
         facet=true in the request, these are the only facets they
         will be able to see counts for; regardless of what other
         facet.field or facet.query params they may specify.

         NOTE: there is *absolutely* nothing a client can do to prevent these
         "invariants" values from being used, so don't use this mechanism
         unless you are sure you always want it.
      -->
    <!--
       <lst name="invariants">
         <str name="facet.field">cat</str>
         <str name="facet.field">manu_exact</str>
         <str name="facet.query">price:[* TO 500]</str>
         <str name="facet.query">price:[500 TO *]</str>
       </lst>
      -->
    <!-- If the default list of SearchComponents is not desired, that
         list can either be overridden completely, or components can be
         prepended or appended to the default list.  (see below)
      -->
    <!--
       <arr name="components">
         <str>nameOfCustomComponent1</str>
         <str>nameOfCustomComponent2</str>
       </arr>
      -->
  </requestHandler>

* Now we have configured our Solr core to work with Blacklight
* PS. remember to reload the core from the web admin page

## Configuring Blacklight

* Now it's time to configure Blacklight
* We need to configure our installation to know how to communicate with Solr and what kind of fields it should expect to see
* Unfortunately no graphical interface here either

In [1]:
## Assuming we are in the folder where we created the Blacklight app in the first place
cd Solr_frontend

In [3]:
cat config/blacklight.yml

# = jetty_path key
# each environment can have a jetty_path with absolute or relative
# (to app root) path to a jetty/solr install. This is used
# by the rake tasks that start up solr automatically for testing
# and by rake solr:marc:index.  
#
# jetty_path is not used by a running Blacklight application
# at all. In general you do NOT need to deploy solr in Jetty, you can deploy it
# however you want.  
# jetty_path is only required for rake tasks that need to know
# how to start up solr, generally for automated testing. 

development:
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:8983/solr/blacklight-core" %>
test: &test
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:#{ENV['TEST_JETTY_PORT'] || 8983}/solr/blacklight-core" %>
production:
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:8983/solr/blacklight-core" %>




* Here we just need to change the port and core name to match ours
* Like so:



In [4]:
cat config/blacklight.yml

# = jetty_path key
# each environment can have a jetty_path with absolute or relative
# (to app root) path to a jetty/solr install. This is used
# by the rake tasks that start up solr automatically for testing
# and by rake solr:marc:index.  
#
# jetty_path is not used by a running Blacklight application
# at all. In general you do NOT need to deploy solr in Jetty, you can deploy it
# however you want.  
# jetty_path is only required for rake tasks that need to know
# how to start up solr, generally for automated testing. 

development:
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:8900/solr/reddit" %>
test: &test
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:#{ENV['TEST_JETTY_PORT'] || 8900}/solr/reddit" %>
production:
  adapter: solr
  url: <%= ENV['SOLR_URL'] || "http://127.0.0.1:8900/solr/reddit" %>


* Now Blacklight can communicate to Solr, but it still doesn't know what kind of fields to expect 

In [9]:
cd app/controllers
pwd

/home/avjves/IR_Course/Solr_frontend/app/controllers


* The file we need to edit now is called catalog_controller.rb 
* It has quite a lot of things in it by default, so we need to remove ( or comment out ) a lot from it

* Starting from the top, first interesting line is:
* <b>config.index.title_field </b>
    * This is by default set to take the id field from your Solr documents, but if you want to identify your documents with some other field,set:
    * ```config.index.title_field = "other_field_name"```
* <b>config.add_facet_field</b>
    * Comment or delete the original lines that start with this
    * This will tell which fields to use as facets
    * Let's say we have a field "subreddit" that we wish to use as a facet, we would then add a line:
    * ```config.add_facet_field 'subreddit', label: 'Subreddit'```
    * Here we tell Blacklight that the Solr field is called subreddit, but we want Blacklight to label it as Subreddit
* <b>config.add_index_field</b>
    * Comment or delete
    * These are the fields that Blacklight will show you when you query for something
    * Similar style as with the facets, first specify the actual field name and then the label
    * ```config.add_index_field 'title_display', label: 'Title'```
* <b>config.add_show_field</b>
    * Comment or delete
    * These are the fields that are shown when you open a more specific info view for one document
    * Same as index fields
    * ```config.add_show_field 'subtitle_display', label: 'Subtitle'```
* <b>config.add_search_field</b>
    * You can specify what kind of queries you want to make, e.g. search for the title only
    * Leave the first one with "All Fields" intact, the rest you can comment out / delete
    * format: <br><br>
       config.add_search_field("text") do |field|<br>
        field.solr_local_parameters = &#123;<br>
         qf: "&#36;text_qf"<br>
        &#125;<br> end <br>
    * So here we create a search field called "text". When querying with this enabled, it also adds the qf part to the query, so we will only search from fields defined in text_qf
* <b>config.add_sort_field</b>
    * Comment or delete
    * Sorting options
    * Specify which field to sort and if it ascending or descending, and a label for it
    * ```config.add_sort_field 'ups desc', label: 'Upvotes'```

In [11]:
##This is the file that controls all fields etc. It has quite a lot of things in it
cat catalog_controller.rb


# frozen_string_literal: true
class CatalogController < ApplicationController

  include Blacklight::Catalog
  include Blacklight::Marc::Catalog


  configure_blacklight do |config|
    ## Class for sending and receiving requests from a search index
    # config.repository_class = Blacklight::Solr::Repository
    #
    ## Class for converting Blacklight's url parameters to into request parameters for the search index
    # config.search_builder_class = ::SearchBuilder
    #
    ## Model that maps search index responses to the blacklight response model
    # config.response_model = Blacklight::Solr::Response

    ## Default parameters to send to solr for all search-like requests. See also SearchBuilder#processed_parameters
    config.default_solr_params = {
      rows: 10
    }

    # solr path which will be added to solr base url before the other solr params.
    #config.solr_path = 'select'

    # items to show per page, each number in the array represent another option to choose from

Once we have edited the controller file, we can go back to the main folder for the project and start the server

In [4]:
cd /home/avjves/IR_course/Solr_frontend
pwd

/home/avjves/Solr_frontend


In [5]:
rails server -p 30001

=> Booting Puma
=> Rails 5.0.2 application starting in development on http://localhost:30001
=> Run `rails server -h` for more startup options
Puma starting in single mode...
* Version 3.8.2 (ruby 2.4.0-p0), codename: Sassy Salamander
* Min threads: 5, max threads: 5
* Environment: development
* Listening on tcp://localhost:30001
Use Ctrl-C to stop
Exiting
